(always be aware of your imports and **be sure to preserve namespaces**!!!)

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy.ndimage as nd

%matplotlib inline

plt.rcParams["image.cmap"] = "gist_gray"

---

### Using a pre-trained Tensorflow model

Training on my laptop's CPU took a long time thus reducing the number of epochs for which I was willing to wait.  Using a GPU, we now have a model trained with 96% accuracy.  Let's use that for our classifier, and try to detect cars in these images:

In [ ]:
# -- get the image path
impath1 = os.path.join("images", "dot_dl", "cctv528_000049.jpg")
impath2 = os.path.join("images", "dot_dl", "cctv679_000024.jpg")

# -- read in the images
img1 = nd.imread(impath1)
img2 = nd.imread(impath2)


# -- plot the images
fig, ax = plt.subplots(1, 2, figsize=(10, 6))
[i.axis("off") for i in ax]
ims = [i.imshow(j) for i, j in zip(ax, (img1, img2))]
fig.canvas.draw()

First things first, let's load the pretrained model and make sure that we're getting high accuracy on a mix of test and training data.

#### LOADING AND TESTING THE PRETRAINED MODEL

First we launch an interactive TF session into which the model can be restored

In [ ]:
sess = tf.InteractiveSession()

Now we restore the model

In [ ]:
# -- define the model path
mdir  = "models"
mpath = os.path.join(mdir, "vehicle_detector.ckpt.meta")

# -- restore it
saver = tf.train.import_meta_graph(mpath)
saver.restore(sess, tf.train.latest_checkpoint(mdir))

then access the graph, the accuracy function, and the input variables for testing,

In [ ]:
# -- access computational graph and utilities for testing
graph     = tf.get_default_graph()
accuracy  = graph.get_tensor_by_name("accuracy:0")
x         = graph.get_tensor_by_name("x:0")
y_        = graph.get_tensor_by_name("y_:0")
keep_prob = graph.get_tensor_by_name("keep_prob:0")

Let's get the postage stamps for testing like we did before,

In [ ]:
# -- get path to images
stpath = os.path.join("images", "dl_training_lum", "*.npy")
stlist = sorted(glob.glob(stpath))
nstamp = len(stlist)

# -- set a shuffle index
np.random.seed(314)
sind = np.random.rand(nstamp).argsort()

# -- read the postage stamps and shuffle
stamps = np.array([np.load(i)[1:-1, 1:-1].flatten() for i in stlist]).astype(np.float32)[sind]
npix   = stamps.shape[1]
nside  = int(np.sqrt(npix))

# -- set the labels
labels = np.array([[1.0, 0.0] if "pos" in i  else [0.0, 1.0] for i in stlist]).astype(np.float32)[sind]

and feed these postage stamps forward through the network:

In [ ]:
# -- set the inputs and feed forward
test_in = {x:stamps, y_:labels, keep_prob:1.0} # don't use dropout for testing
test_accuracy = accuracy.eval(feed_dict=test_in)
print("test accuracy {0}".format(test_accuracy))

(notice the accuracy is **higher** than the testing accuracy when we built the model! ... why might that be?)

---

#### DETECTING VEHICLES IN A WHOLE IMAGE

We trained the model on postage stamps (which means that the model is expecting postage stamp size ($48 \times 48$ pixels) inputs.  Thus, we need to break our input up into many such sub images and keep track of the locations of these subimages.  This process is sometimes known as the "sliding window" approach.

First we convert our images to scaled luminosities:

In [ ]:
# -- convert to luminosity
img1_L = img1.mean(-1) / 255.
img2_L = img2.mean(-1) / 255.

Now we break up our image into $48 \times 48$ postage stamps with some indexing:

In [ ]:
# -- we'll want our image to contain an integer number of postage stamps
buff   = 8
img1_L = img1_L[:, buff:-buff]
img2_L = img2_L[:, buff:-buff]

How many postage stamps does this make?  Well, since our postage stamp size is $48 \times 48$, this means that

In [ ]:
# -- calculate the number of postage stamps per image
nrow, ncol = img1_L.shape
nst_r = nrow - 48
nst_c = ncol - 48
print("total number of postage stamps in this image is {0}".format(nst_r * nst_c))

That's a **lot** of postage stamps... we'll reduce this number in a minute.  First let's set up the indexing to get the postage stamps.

In [ ]:
# -- set up row and column indices
rind = np.arange(nrow * ncol).reshape(nrow, ncol) // ncol
cind = np.arange(nrow * ncol).reshape(nrow, ncol) % ncol

rlo = rind[:-48, 24:-24].flatten()
rhi = rind[48:, 24:-24].flatten()
clo = cind[24:-24, :-48].flatten()
chi = cind[24:-24, 48:].flatten()

# -- pull off postage stamps
stamps1 = np.array([img1_L[i:j, k:l] for i, j, k, l in zip(rlo, rhi, clo, chi)]).reshape(nst_r * nst_c, npix)
stamps2 = np.array([img2_L[i:j, k:l] for i, j, k, l in zip(rlo, rhi, clo, chi)]).reshape(nst_r * nst_c, npix)

In [ ]:
# -- verify that these are the right size
print("number of postage stamps == 55296? : {0}".format(stamps1.shape[0] == nst_r * nst_c))

Let's plot and make sure

In [ ]:
# -- plot the images
fig1, ax1 = plt.subplots(figsize=(10, 6))
ax1.axis("off")
ax1.imshow(img1_L)
fig1.canvas.draw()

fig2, ax2 = plt.subplots(1, 2, figsize=(8.25, 5))
[i.axis("off") for i in ax2]
ims = [i.imshow(j) for i, j in zip(ax2, (stamps1[0].reshape(48, 48), stamps1[-1].reshape(48, 48)))]
fig2.canvas.draw()

For each of these postage stamps, we want to know the probability that there is a vehicle.  Recall from the model building that the output layer was called "y_conv".  Let's calculate this for some postage stamps (with "strides" set to 3 in the row and column directions).

In [ ]:
# -- calculate index for 3x3 strides and sub-select postage stamps
gind = np.arange(nst_r * nst_c).reshape(nst_r, nst_c)[::3, ::3].flatten()
sub  = stamps1[gind]

# -- pass these through the network
y_conv = graph.get_tensor_by_name("y_conv:0")
indict = {x:sub, keep_prob:1.0}
output = sess.run(y_conv, indict)

In [ ]:
print("CNN output = \n{0}".format(output))

Let's apply our own softmax to the output:

In [ ]:
# -- apply softmax
prob = np.exp(output) / np.exp(output).sum(1, keepdims=True)

In [ ]:
# -- display
detect = prob[:, 0].reshape(nst_r/3, nst_c/3) > 0.99
img1_L_reg = img1_L[24:-24, 24:-24]

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
[i.axis("off") for i in ax]
ims = [i.imshow(j) for i, j in zip(ax, (img1_L_reg, detect))]
fig.canvas.draw()

And the second image:

In [ ]:
# -- pass postage stamps from second image through
indict2 = {x:stamps2[gind], keep_prob:1.0}
output2 = sess.run(y_conv, indict2)

In [ ]:
# -- apply softmax
prob2 = np.exp(output2) / np.exp(output2).sum(1, keepdims=True)

In [ ]:
# -- display
detect2 = prob2[:, 0].reshape(nst_r/3, nst_c/3) > 0.99
img2_L_reg = img2_L[24:-24, 24:-24]

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
[i.axis("off") for i in ax]
ims = [i.imshow(j) for i, j in zip(ax, (img2_L_reg, detect2))]
fig.canvas.draw()

What about a random image?

In [ ]:
img3 = nd.imread(os.path.join("images", "cctv734_ex.jpg"))

# -- plot the images
fig, ax = plt.subplots(figsize=(6, 6))
ax.axis("off")
im = ax.imshow(img3)
fig.canvas.draw()

In [ ]:
# -- pull postage stamps from image
img3_L = img3[:, buff:-buff].mean(-1) / 255.
stamps3 = np.array([img3_L[i:j, k:l] for i, j, k, l in zip(rlo, rhi, clo, chi)]).reshape(nst_r * nst_c, npix)

In [ ]:
# -- feed postage stamps through network
indict3 = {x:stamps3[gind], keep_prob:1.0}
output3 = sess.run(y_conv, indict3)

In [ ]:
# -- apply softmax
prob3 = np.exp(output3) / np.exp(output3).sum(1, keepdims=True)

In [ ]:
# -- display
detect3 = prob3[:, 0].reshape(nst_r/3, nst_c/3) > 0.99
img3_L_reg = img3_L[24:-24, 24:-24]

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
[i.axis("off") for i in ax]
ims = [i.imshow(j) for i, j in zip(ax, (img3_L_reg, detect3))]
fig.canvas.draw()

---

**PRACTICE**: Notice that we built the network using only the luminosity of the pixels.  We could have used all three color channels and may have gotten a more accurate classifier.  How would the network need to change to incorporate all three color channels?  I have included RGB versions of the postage stamps in the folder image/dl_training_scl.  Give it a try! 